# Assignment on Programming Techniques for Big Data

Functional programming is the basis of most modern Big Data processing systems.
Before going forward to the course, it is important to master data processing
techniques using a functional programming style. In this assignment, your task 
is to train yourselves in thinking in a functional way when processing data. 

Many of the the tasks below are easy, but some are not and require many
iterations (and extensive testing!) to get right. For some of them, you
can find ready-made solutions on the internet. Even though it may be tempting,
I advise you against copying and pasting them here, as you will regret it
later on in the course.


This assignment has a total of *115* points. Your grade is calculated with `min(points/10, 10)`, i.e. you only need 100 points for a 10. A few notes:

* For each function you define, you also need to define at least one working example.
* Do not change the given function signatures.
* Use functional programming for every question (besides the first section).

## Foundations of functional programming

In this part you will implement core functions that are vital to functional programming.

**T** (5pts): Implement `map` using iteration for lists/arrays

In [ ]:
from inspect import signature

In [ ]:
def map(func, xs):
  for item in xs:
    yield func(item)
    
list(map(lambda x: x*2, range(7)))


In [ ]:
def map(func, xs):
  return (func(x) for x in xs)
    
list(map(lambda x: x*2, range(7)))

**T** (5pts): Implement `filter` using iteration for lists/arrays

In [ ]:
def filter(func, xs):
  for item in xs:
    if func(item):
      yield item

list(filter(lambda x: x % 2 == 0, range(7)))

In [ ]:
def filter(func, xs):
  return (x for x in xs if func(x))

list(filter(lambda x: x % 2 == 0, range(7)))

**T** (5pts): Implement `reduceR` using iteration for lists/arrays

In [ ]:
def reduceR(func, xs, init):
  if len(xs) == 0:
    if len(signature(func).parameters) < 2:
      return func(init)
    else:
      return init
  result = init
  for item in xs:
    result = func(item, result)
  return result

reduceR(lambda x, y: x-y, range(7), 0)

**T** (5pts): Implement a function `flatten(xs: [[A]]): [A]` that converts a list of 
lists into a list formed by the elements of these lists. For example:

```python
>>> a = [[1,2],[2,3],[3,[4]]]
>>> flatten(a)
[1,2,2,3,3,[4]]
```

In [ ]:
def flatten(xss):
  for item in xss:
    for sub_item in item:
      yield sub_item


list(flatten([[1,2,3],[4,5], [7,[8,9]]]))

In [ ]:
def flatten(xss):
  return (sub_item for item in xss for sub_item in item)


list(flatten([[1,2,3],[4,5], [7,[8,9]]]))

## More basic functions

In every implementation from now (also in next steps)on you should reuse at least one of your answers to an earlier question.

**T** (5pts): Implement `reduceL` by reusing `reduceR`

In [ ]:
def reduceL(func, xs, init):
  return reduceR(lambda x,y:func(y,x), xs[::-1], init)


reduceL(lambda x, y: x-y, range(7),0)

**T** (10pts): Implement `group_by` by reusing `reduceL`.

In [ ]:
def group_by(classifier, xs):
  return reduceL(lambda x,y: {**x,**{classifier(y):[*x.get(classifier(y),[]),y]}}  ,xs,{})
group_by(lambda x: "even" if x % 2 == 0 else "odd", range(10))

## Simple data processing



**T** (5pts): Implement `distinct` using `reduceL`. Also provide its function signature.

In [ ]:
# distinct(xs:[A]): [A]
def distinct(xs):
    yield from set(xs)

a = [1,2,3,1,2,3,4,5,6,5,4,3,2,1]
list(distinct(a))

**T** (5pts): Implement `flatmap` and provide its function signature.

In [ ]:
# flatmap(func: A -> [B], [A]): [B]
def flatmap(func, xs):
  return flatten(map(func,xs))

list(flatmap(lambda x: range(x), range(5)))

**T** (5pts): Implement `max(xs: [Integer]): Integer` that finds the largest value in `xs`. You can assume the list is non-empty.

In [ ]:
def max(xs):
  max_item = xs[0]
  for i in xs:
    if i > max_item:
      max_item = i
  return max_item

max([1,59,42,27,38])

In [ ]:
def max(xs):
  return reduceL(lambda x,y:x if x>y else y,xs,xs[0])

max([1,59,42,27,38])

## Higher-order functions


**T** (10pts): Implement a function called `drop_while(f: A -> Boolean, xs: [A]) : [A]` 
that drops the longest prefix of elements from `xs` that satisfy `f`.

```python
>>> a = [1,2,3,4,5,6]
>>> dropWhile(lambda x: x <= 3, a)
[4,5,6]
```

In [ ]:
def drop_while(func, xs):
  xs = iter(xs)
  for x in xs:
    if not func(x):
        yield x
        break
  for x in xs:
    yield x


list(drop_while(lambda x: x <= 3, [1,2,1,3,5,3,1,4,1,5,6]))

**T** (10pts): Implement a function `zip(xs: [A], ys: [B]): List[(A,B)]` that returns a list formed from this list and another list by combining the corresponding elements in pairs. If one of the two lists is longer 
than the other, its remaining elements are ignored. 

```python
>>> a = [1,2,3,4]
>>> b = [a,b,c,d,e]
>>> zip(a,b)
[(1, 'a'), (2, 'b'), (3, 'c'), (4,'d')]
```

In [ ]:
def zip(xs, ys):
  result_list = []
  for i in range(min([len(xs),len(ys)])):
    result_list.append((xs[i],ys[i]))
  yield from result_list
      

    
a = [2,3,4]
b = ['a','b','c','d']
list(zip(a,b))

In [ ]:
def min(xs):
    return reduceL(lambda x,y:x if x<y else y,xs,xs[0])

def zip(xs, ys):
  return map(lambda i:(xs[i],ys[i]),range(min([len(xs),len(ys)])))
      

    
a = [2,3,4]
b = ['a','b','c','d']
list(zip(a,b))

**T** (10pts): Implement a function 
`scanL(f: (acc: B, x: A) -> B, xs: [A], init: B) -> [B]`
that works like `reduceL` but instead of producing one final result, it also
returns all the intermediate results.

```python
>>> a = [2,3,4]
>>> scanL(a, 0, lambda x, y: x + y)
[0, 2, 5, 9]
```

In [ ]:
def scanL(func, xs, init):
  result_list = [init]
  for i in range(len(xs)):
    result_list.append(reduceL(func,xs[:i+1],init))
  return result_list
    
scanL(lambda x, y: x + y, [2,3,4], 0)

In [ ]:
def scanL(func, xs, init):
  yield init
  for x in xs:
    init = func(init, x)
    yield init
list(scanL(lambda x, y: x + y, [2,3,4], 0))

In [ ]:
import functools
import operator

def foldl(func, acc, xs):
  return functools.reduce(func, xs, acc)
foldl(lambda x, y: x-y,0, range(7))

In [ ]:
import functools
import operator

def flip(func):
    @functools.wraps(func)
    def newfunc(x, y):
        return func(y, x)
    return newfunc

def foldr(func, acc, xs):
    return functools.reduce(flip(func), reversed(xs), acc)

foldr(lambda x, y: x-y,0, range(7))